# Using an HLS core in PYNQ

In this notebook we will finally interact with the HLS Core we wrote in [Building a Bitstream](3-Building-A-Bitstream.ipynb)


## Outputs from **[Building a Bitstream](3-Building-A-Bitstream.ipynb)**

The first two critical components of a PYNQ overlay are a `.tcl` script file and a bitfile. These files should have been created in **[Building a Bitstream](3-Building-A-Bitstream.ipynb)** and with the names `io.tcl` and `io.bit`.

**You can skip this step by running the command below:**

In [ ]:
!cp /home/xilinx/PYNQ-HLS/pynqhls/io/io.tcl /home/xilinx/PYNQ-HLS/tutorial/pynqhls/io/
!cp /home/xilinx/PYNQ-HLS/pynqhls/io/io.bit /home/xilinx/PYNQ-HLS/tutorial/pynqhls/io/

Otherwise, verify that these files are in the `~/PYNQ-HLS/tutorial/pynqhls/io` folder of your PYNQ-HLS repository on your **host computer** by running the following commands from Cygwin, or a Bash Terminal.

```bash
    ls ~/PYNQ-HLS/tutorial/pynqhls/io/io.tcl
    ls ~/PYNQ-HLS/tutorial/pynqhls/io/io.bit
```
   
Using [SAMBA](http://pynq.readthedocs.io/en/v2.0/getting_started.html#accessing-files-on-the-board), or SCP, copy these files from your host machine to the directory `/home/xilinx/PYNQ-HLS/tutorial/pynqhls/io/` on your PYNQ board.

Verify that these files are there by running the following cells: 

In [ ]:
!ls /home/xilinx/PYNQ-HLS/tutorial/pynqhls/io/io.tcl

In [ ]:
!ls /home/xilinx/PYNQ-HLS/tutorial/pynqhls/io/io.bit

## Python Files

Before we verify that the `io.tcl` and `io.bit` files are working correctly, we need to create the Python files that complete our PYNQ Overlay. Two files are required: 

1. `__init__.py` The Python file that defines an importable Python package
2. `io.py` The Python class that interacts with the FPGA bitstream

**To skip this step you can run the following cell: **

In [ ]:
!cp /home/xilinx/PYNQ-HLS/pynqhls/io/io.py         /home/xilinx/PYNQ-HLS/tutorial/pynqhls/io/
!cp /home/xilinx/PYNQ-HLS/pynqhls/io/__init__.py   /home/xilinx/PYNQ-HLS/tutorial/pynqhls/io/

Otherise follow these instructions:

### `__init__.py`

`__init__.py` is simple, so we will start there. This file defines an importable Python package. 

Copy the following cell into a file named `__init__.py` in the `/home/xilinx/PYNQ-HLS/tutorial/pynqhls/io/` directory on your PYNQ board. 

In [ ]:
from .io import ioOverlay

This declares the `ioOverlay` Python class to be part of the `io` package. By residing in the `pynqhls` folder, it is part of the `pynqhls` package, which has its own `__init__.py` file. You can view the contents of that file by executing the cell below: 

In [ ]:
!cat /home/xilinx/PYNQ-HLS/tutorial/pynqhls/__init__.py

### `io.py`

Next, we create the `io.py` file that defines the `ioOverlay` class as an interface for our FPGA Bitstream.

Copy and paste the following cell into a file named `io.py` in the `/home/xilinx/PYNQ-HLS/tutorial/pynqhls/io/` directory on your PYNQ board. 

This code is analyzed in subsequent cells. 

In [ ]:
from pynq import Overlay, GPIO, Register, MMIO
import os
import inspect
import numpy as np
class ioOverlay(Overlay):
    """A simple Physical IO Overlay for PYNQ.

    This overlay is implemented with a single CtrlLoop core
    connected directly to the ARM Core AXI interface.

    """
    __RESET_VALUE = 0
    __NRESET_VALUE = 1

    """ For convenince, we define register offsets that are scraped from
    the HLS implementation header files.

    """
    __IO_AP_CTRL_OFF = 0x00
    __IO_AP_CTRL_START_IDX = 0
    __IO_AP_CTRL_DONE_IDX  = 1
    __IO_AP_CTRL_IDLE_IDX  = 2
    __IO_AP_CTRL_READY_IDX = 3
    __IO_AP_CTRL_AUTORESTART_IDX = 7

    __IO_GIE_OFF     = 0x04
    __IO_IER_OFF     = 0x08
    __IO_ISR_OFF     = 0x0C
    
    """These define the 'reg' argument to the 'io' HLS function.  The
    memory space defined here is shared between the HLS core and the
    ARM PS.

    """
    __IO_REG_OFF = 0x200
    __IO_REG_LEN = 0x100
    def __init__(self, bitfile, **kwargs):
        """Initializes a new ioOverlay object.

        """
        # The following lines do some path searching to enable a 
        # PYNQ-Like API for Overlays. For example, without these 
        # lines you cannot call ioOverlay('io.bit') because 
        # io.bit is not on the bitstream search path. The 
        # following lines fix this for any non-PYNQ Overlay
        #
        # You can safely reuse, and ignore the following lines
        #
        # Get file path of the current class (i.e. /opt/python3.6/<...>/stream.py)
        file_path = os.path.abspath(inspect.getfile(inspect.currentframe()))
        # Get directory path of the current class (i.e. /opt/python3.6/<...>/stream/)
        dir_path = os.path.dirname(file_path)
        # Update the bitfile path to search in dir_path
        bitfile = os.path.join(dir_path, bitfile)
        # Upload the bitfile (and parse the colocated .tcl script)
        super().__init__(bitfile, **kwargs)
        # Manually define the GPIO pin that drives reset
        self.__resetPin = GPIO(GPIO.get_gpio_pin(0), "out")
        # Define a Register object at address 0x0 of the IO address space
        # We will use this to set bits and start the core (see start())
        # Do NOT write to __ap_ctrl unless __resetPin has been set to __NRESET_VALUE
        self.nreset()
        self.__ap_ctrl = Register(self.ioCore.mmio.base_addr, 32)
        self.__hls_reg = MMIO(self.ioCore.mmio.base_addr + self.__IO_REG_OFF,
                              self.__IO_REG_LEN)

    def __set_autorestart(self):
        """ Set the autorestart bit of the HLS core
        """
        self.__ap_ctrl[self.__IO_AP_CTRL_AUTORESTART_IDX] = 1

    def __clear_autorestart(self):
        """ Clear the autorestart bit
        """
        self.__ap_ctrl[self.__IO_AP_CTRL_AUTORESTART_IDX] = 0

    def __start(self):
        """Raise AP_START and enable the HLS core

        """
        self.__ap_ctrl[self.__IO_AP_CTRL_START_IDX] = 1

    def __stop(self):
        """Lower AP_START and disable the HLS core

        """
        self.__ap_ctrl[self.__IO_AP_CTRL_START_IDX] = 0

    def nreset(self):
        """Set the reset pin to self.__NRESET_VALUE to place the core into
        not-reset (usually run)

        """
        self.__resetPin.write(self.__NRESET_VALUE)
        
    def reset(self):
        """Set the reset pin to self.__RESET_VALUE to place the core into
        reset

        """
        self.__resetPin.write(self.__RESET_VALUE)

    def launch(self):
        """ Start and detatch computation on the io HLS core
        
        Returns
        -------
        Nothing
        
        """        
        self.__set_autorestart()
        self.__start()
        return
        
    def land(self):
        """ Re-Connect and Terminate Computation on the io HLS core
        
        Returns
        -------
        The 4-bit value representing the value of the buttons.
        
        """
        self.__clear_autorestart()
        while(not self.__ap_ctrl[self.__IO_AP_CTRL_DONE_IDX]):
            pass
        self.__stop()
        return self.__hls_reg.read(0)

    def run(self):
        """ Launch computation on the io HLS core
        
        Returns
        -------
        The 4-bit value representing the value of the buttons.
        
        """
        self.__start()
        while(not self.__ap_ctrl[self.__IO_AP_CTRL_DONE_IDX]):
            pass
        self.__stop()
        return self.__hls_reg.read(0)

The code above defines a the `ioOverlay` class for interacting with the bitfile we've created - **don't be scared by the length, much of it is comments**! 

#### `__init__`

The class begins with an `__init__` method. The first lines in `__init__` method add the class directory to the bitstream search path, so that a bitstream can be loaded using the relative path (e.g. `io.bit`) instead of the absolute path (e.g. `/opt/python3.6/lib/python3.6/site-packages/pynqhls/io/io.bit`). The last lines in `__init__` define the reset pin as a GPIO object, and a `Register` object that points to the address of the HLS core.

If you haven't seen the `Register` class before - it is quite useful. It allows you to read and set bits of a single memory location using array indicies. More documentation can be found on the [PYNQ Read The Docs Page](http://pynq.readthedocs.io/en/v2.0/pynq_package/pynq.ps.html#pynq.ps.Register).

For example, if there is a register at the address `0xdeadbeef`, you can use the Register class to maniuplate it

``` python
    foo = Register(0xbeefcafe, 32)
    foo[31:8] = 0xc0ffee
```
Thus bits 31 to 8 of the address `0xbeefcafe` are set to `0xc0ffee`

The offset constants used above are defined in `xctrlloop_hw.h` which is generated by Vivado HLS. This file is shown below: 

``` C

// CTRL
// 0x000 : Control signals
//         bit 0  - ap_start (Read/Write/COH) - __IO_AP_CTRL_START_IDX in io.py
//         bit 1  - ap_done (Read/COR) - __IO_AP_CTRL_DONE_IDX in io.py
//         bit 2  - ap_idle (Read) - __IO_AP_CTRL_IDLE_IDX in io.py
//         bit 3  - ap_ready (Read) - __IO_AP_CTRL_READY_IDX in io.py
//         bit 7  - auto_restart (Read/Write) - __IO_AP_CTRL_AUTORESTART_IDX in io.py
//         others - reserved
// 0x004 : Global Interrupt Enable Register
//         bit 0  - Global Interrupt Enable (Read/Write)
//         others - reserved
// 0x008 : IP Interrupt Enable Register (Read/Write)
//         bit 0  - Channel 0 (ap_done)
//         bit 1  - Channel 1 (ap_ready)
//         others - reserved
// 0x00c : IP Interrupt Status Register (Read/TOW)
//         bit 0  - Channel 0 (ap_done)
//         bit 1  - Channel 1 (ap_ready)
//         others - reserved
// 0x200 ~
// 0x3ff : Memory 'regs_V' (128 * 32b)
//         Word n : bit [31:0] - regs_V[n]
// (SC = Self Clear, COR = Clear on Read, TOW = Toggle on Write, COH = Clear on Handshake)

#define XCTRLLOOP_CTRL_ADDR_AP_CTRL     0x000 // __IO_AP_CTRL_OFF in io.py
#define XCTRLLOOP_CTRL_ADDR_GIE         0x004 // __IO_GIE_OFF in io.py
#define XCTRLLOOP_CTRL_ADDR_IER         0x008 // __IO_IER_OFF in io.py
#define XCTRLLOOP_CTRL_ADDR_ISR         0x00c // __IO_ISR_OFF in io.py
#define XCTRLLOOP_CTRL_ADDR_REGS_V_BASE 0x200 // __IO_REG_OFF in io.py
#define XCTRLLOOP_CTRL_ADDR_REGS_V_HIGH 0x3ff
#define XCTRLLOOP_CTRL_WIDTH_REGS_V     32
#define XCTRLLOOP_CTRL_DEPTH_REGS_V     128   // __IO_REG_LEN in io.py

```

Following `__init__` there are several methods for operating the overlay:


#### `__set_autorestart` / `__clear_autorestart`

Every HLS core has a *autorestart* control bit that re-runs the HLS core when it finishes execution. This bit is at index 7 (`__IO_AP_CTRL_AUTORESTART_IDX`) of the HLS Control Register (`__IO_AP_CTRL_OFF`). Setting this bit wraps the HLS core in an implicit `while(autorestart)` loop that only terminates when the autorestart bit is cleared. The `__set_autorestart` method sets the autorestart bit, causing the HLS core to loop when started until `__clear_autorestart` is called. 

#### `__start` / `__stop`

The `_start` method sets the *start* control bit, causing the HLS core to begin computation, and the `_stop` method clears it. This bit is at index 0 (`__IO_AP_CTRL_START_IDX`) of the HLS Control Register (`__IO_AP_CTRL_OFF`).

#### `reset` / `nreset`

The `reset` method asserts the GPIO Pin at Index 0 to reset the HLS core. This was connected to the `userReset` core in **[Building a Bitstream](3-Building-A-Bitstream.ipynb)**. The `nreset` method does the opposite.

#### `launch` / `land`

The `launch` method sets the autorestart bit, starts computation on the HLS core, and then returns. Conceptually this launches the HLS core as a separate thread that does not terminate. To terminate the user must call the `land` method, which clears the autorestart bit, and stops computation on the HLS core. 

#### `run`

The `run` method starts computation on the HLS core and waits for the HLS core to terminate by checking the *done* bit at index 1 (`__IO_AP_CTRL_DONE_IDX`) of the HLS Control Register (`__IO_AP_CTRL_OFF`). This means that the HLS core runs once, as if it is a software method.

More information about the HLS Control registers can be found in the [HLS User Guide](https://www.xilinx.com/support/documentation/sw_manuals/xilinx2017_1/ug902-vivado-high-level-synthesis.pdf)


## Interacting with the IO Overlay

Once the `__init__.py` and `io.py` files are in place with `io.bit` and `io.tcl`, we can use the overlay.

The following cell adds the PYNQ-HLS repository to the Python Package search path. Once we install the overlay, this cell will not be needed:

In [ ]:
import sys
sys.path.insert(0, '/home/xilinx/PYNQ-HLS/tutorial/')

Like in previous examples, load the PYNQ Overlay: 

In [ ]:
from pynqhls.io import ioOverlay
overlay = ioOverlay('io.bit')

First, we will test the `run` method. Press and hold one of the push buttons on the PYNQ board while executing the following cell. The cell will light the first LED, print a message on the UART pins, and print a value representing the one-hot encoding of the value of the buttons. 

In [ ]:
buttons = overlay.run()
for i in range(4):
    if(buttons & (1<<i)):
        print(f"Button {i} is pressed!")

Next, we will test the `launch` and `land` methods. The following cell will execute for 10 seconds. While the cell is executing, the LEDs on the PYNQ board will count upward. 

In [ ]:
import time
overlay.launch()
time.sleep(10)
buttons = overlay.land()
for i in range(4):
    if(buttons & (1<<i)):
        print(f"Button {i} is pressed!")

And that's it! 

In the **[Packaging an Overlay](5-Packaging-an-Overlay.ipynb)** notebook we will make a Python Installation script!